# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# Get the data
data = pd.read_csv('/Users/nferrari/Documents/IronHack/Git-repo/lab-dataframe-calculations/your-code/Orders.csv')
data

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


In [4]:
# Get dataset by country
country = pd.DataFrame(data['Country'],data['CustomerID'])
country = country.reset_index()
country = country.drop_duplicates('CustomerID')
country

,CustomerID,Country
0,17850,United Kingdom
9,13047,United Kingdom
26,12583,United Kingdom
46,13748,United Kingdom
65,15100,United Kingdom
...,...,...
394965,13436,United Kingdom
395041,15520,United Kingdom
395777,13298,United Kingdom
396506,14569,United Kingdom


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [ ]:
# aggregate amount spend by unique customer
# Get the total amount spent per client
data2 = data['amount_spent'].groupby(by= data['CustomerID'], axis=0, dropna=True).sum()
data2 = pd.DataFrame(data2)
data2 = data2.reset_index()
data2

In [5]:
# select customers aggregated amount spent in quantile range
percentiles = data2.describe(percentiles=[ .75, .95])
percentiles = percentiles.drop(['CustomerID'],axis=1)
percentiles.columns = ['Threshold Amount for given quantiles']
percentiles = percentiles.T
percentiles = percentiles.drop(['count', 'mean' , 'std', 'min', '50%', 'max'],axis=1)
percentiles

,75%,95%
Threshold Amount for given quantiles,1661.64,5840.182


In [6]:
# label customers as VIP or Preferred
def quantile_class(row):
    if row['amount_spent'] >= np.percentile(data2['amount_spent'], 95) :
        return 'VIP'
    elif row['amount_spent'] <= np.percentile(data2['amount_spent'], 75):
        return np.nan
    else:
        return 'Preferred'

In [7]:
# apply function no label customers
data2['Client Class'] = data2.apply(quantile_class, axis =1)
data2 = data2.dropna()
data2

,CustomerID,amount_spent,Client Class
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
5,12352,2506.04,Preferred
...,...,...,...
4319,18259,2338.60,Preferred
4320,18260,2643.20,Preferred
4328,18272,3078.58,Preferred
4337,18283,2094.88,Preferred


In [8]:
# Merging countries and creating a new dataset

data3 = data2.merge(country, left_on='CustomerID',right_on='CustomerID')
data3.sample(25)

,CustomerID,amount_spent,Client Class,Country
309,13777,25977.16,VIP,United Kingdom
1064,18198,5425.56,Preferred,United Kingdom
107,12683,8476.57,VIP,Norway
823,16754,2002.40,Preferred,United Kingdom
698,15981,3408.48,Preferred,United Kingdom
296,13704,2308.06,Preferred,United Kingdom
397,14227,3350.96,Preferred,United Kingdom
862,17017,8694.26,VIP,United Kingdom
83,12613,3344.38,Preferred,United Kingdom
285,13644,2624.26,Preferred,United Kingdom


In [9]:
# Change CustomerID to a categorical to avoid calculations
data3['CustomerID'] = data3['CustomerID'].astype(object)
data3.dtypes

CustomerID       object
amount_spent    float64
Client Class     object
Country          object
dtype: object

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [10]:
# Filtering VIP customers
vips_selection=data3[data3['Client Class'] == 'VIP']
vips_selection.sort_values(by='Country')

,CustomerID,amount_spent,Client Class,Country
595,15311,60767.90,VIP,EIRE
218,13225,6463.84,VIP,EIRE
589,15290,7943.22,VIP,EIRE
216,13209,5891.61,VIP,France
169,12971,11189.91,VIP,France
...,...,...,...,...
361,14057,6217.96,VIP,United Kingdom
362,14060,6140.42,VIP,United Kingdom
363,14062,10464.85,VIP,United Kingdom
367,14096,65164.79,VIP,United Kingdom


In [11]:
print('The country that has the most VIP Customers is the', vips_selection['Country'].max())

The country that has the most VIP Customers is the United Kingdom


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [12]:
# Filtering both VIP and Preferred label
vips_preferred=data3[(data3['Client Class'] == 'VIP') | (data3['Client Class'] == 'Preferred')]
vips_preferred.sort_values(by='Country')

,CustomerID,amount_spent,Client Class,Country
198,13126,2099.64,Preferred,Channel Islands
197,13124,3866.33,Preferred,Channel Islands
204,13141,2388.75,Preferred,Denmark
203,13139,5064.94,Preferred,Denmark
202,13137,3650.41,Preferred,Denmark
...,...,...,...,...
410,14285,1910.01,Preferred,United Kingdom
411,14286,4628.75,Preferred,United Kingdom
412,14290,2314.65,Preferred,United Kingdom
399,14235,3144.86,Preferred,United Kingdom
